# Exercício em Python - Tema 8 - Mudanças Climáticas

O objetivo deste exercício é comparar as séries observadas com os sinais projetados por modelos CMIP5/CMIP6.

## 1.Importação de Bibliotecas

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import requests, io

In [ ]:
'''import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, io
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import seaborn as sns'''

## 2. Importação da Base de Dados

In [31]:
modelList = ['ACCESS-CM2','ACCESS-ESM1-5','CMCC-ESM2','EC-EARTH3',
             'GFDL-CM4','GFDL-ESM4','HadGEM3-GC31-LL','INM-CM4_8',
             'INM-CM5','IPSL-CM6A-LR','KACE','KIOST','MIROC6','MPI-ESM1-2',
             'MRI-ESM2','NESM3','NorESM2-MM','TaiESM1','UKESM1-0-LL']

scenList = ['hist','ssp245','ssp585']
# Importa base de dados para cada cenário
for scen in scenList:
    # Monta a URL do arquivo já consolidado para o cenário
    url = f"https://raw.githubusercontent.com/hydrologywatersecurity/livro-python-hidrologia/refs/heads/main/CLIMBra_dataset_1/pr-CABra1-{scen}.csv"
    
    # Faz o download do arquivo
    r = requests.get(url).content
    
    # Lê em dataframe
    df = pd.read_csv(io.StringIO(r.decode('utf-8')))
    
    # Salva localmente (se quiser)
    df.to_csv(f'pr-CABra1-{scen}.csv')
    
    print(f"Arquivo {scen} importado com sucesso. Dimensões: {df.shape}")

Arquivo hist importado com sucesso. Dimensões: (0, 1)
Arquivo ssp245 importado com sucesso. Dimensões: (0, 1)
Arquivo ssp585 importado com sucesso. Dimensões: (0, 1)


In [ ]:
scenList = ['hist','ssp245','ssp585']
colorList  = ['gray','darkgoldenrod','firebrick']

fig, axes = plt.subplots(figsize = (8,4))

for i, scen in enumerate(scenList):
    # Lendo arquivo 
    name = f'pr-CABra1-{scen}.csv'
    df = pd.read_csv(name, parse_dates=['time'], index_col='time')
    
    # Calculando a precipitação total anual
    df_sum = df.resample("YE").sum()
    
    # Calculando estatísticas do ensemble
    ensemble_mean = df_sum.mean(axis=1)
    ensemble_std  = df_sum.std(axis=1)
    
    # Plotando ensemble mean
    plt.plot(ensemble_mean.index, ensemble_mean.values,  marker = 'o',markersize = 3,
             color=colorList[i], linewidth=2)
    
    # Faixa de incerteza (±1 std)
    plt.fill_between(ensemble_mean.index, 
                     ensemble_mean - ensemble_std, 
                     ensemble_mean + ensemble_std, 
                     color=colorList[i], alpha=0.2)

# Formatação do eixo x (anos)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

plt.xlabel("Ano")
plt.ylabel("Precipitação total anual (mm)")
# plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
'''# Bacias escolhidas representando diferentes biomas/regionais hidrográficas
bacias = ['15', '411', '195', '443', '607']
nomes_bacias = {
    '15': 'Amazônia',
    '411': 'Cerrado',
    '195': 'Caatinga',
    '443': 'Mata Atlântica',
    '607': 'Pampa'
}

variaveis = {
    'tmin': 'Temperatura Mínima (C)',
    'tmax': 'Temperatura Máxima (C)',
    'p': 'Precipitação (mm)',
    'pet_pm': 'Evapotranspiração Potencial (mm)'
}

# Dicionário para armazenar os DataFrames por bacia
dfs = {}'''

In [ ]:
'''# Etapa 1: Importação e agregação anual sem função
i = 0
for b in bacias:
    url = f"https://raw.githubusercontent.com/hydrologywatersecurity/livro-python-hidrologia/refs/heads/main/CABRA_dataset/CABra_climate_daily_series/ens/CABra_{b}_climate_ENS.txt"
    r = requests.get(url).content
    df = pd.read_csv(io.StringIO(r.decode('utf-8', errors='ignore')), skiprows=13, delimiter='\t')
    df.columns = ['Year', 'Month', 'Day', 'p', 'tmin', 'tmax', 'rh', 'wnd', 'srad', 'et', 'pet_pm','pet_pt', 'pet_hg']
    #Exclui linha 0 pois não faz parte dos dados
    df = df.drop([0], axis=0)
    #Passando todos os dados para categoria numérica
    df = df.apply(pd.to_numeric)
    #Definir coluna de data
    df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    df['year'] = df['date'].dt.year
    df_ano = df.groupby('year')[['tmin', 'tmax', 'p', 'pet_pm']].mean().reset_index()
    dfs[b] = df_ano
    i += 1'''

##3. Gráfico de comparação das variações climáticas (1980 - 2010)

In [ ]:
'''# Etapa 2: Gráficos de comparação
for var, nome_var in variaveis.items():
    plt.figure(figsize=(12, 6))
    for b in bacias:
        df = dfs[b]
        # Ensure there are non-null values before plotting
        if not df.empty:
            plt.plot(df['year'], df[var], label=f"{nomes_bacias[b]}", marker='o')
    plt.title(f"{nome_var} Anual nas Bacias CABra (1980–2010)", fontsize=14)
    plt.xlabel('Ano')
    plt.ylabel(nome_var)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()'''

##4. Gráfico de comparação das variações climáticas com linhas de tendência (1980 - 2010)

In [ ]:
'''# Etapa 3 (opcional): Adicionar tendência linear
for var, nome_var in variaveis.items():
    plt.figure(figsize=(12, 6))
    for b in bacias:
        df = dfs[b]
        # Ensure there are enough non-null values for linear regression
        if not df.empty and len(df) > 1:
            X = df['year'].values.reshape(-1, 1)
            y = df[var].values
            # Filter out NaN values for linear regression
            not_nan_mask = ~np.isnan(y)
            X_filtered = X[not_nan_mask]
            y_filtered = y[not_nan_mask]

            if len(X_filtered) > 1: # Ensure there are at least two points to fit a line
                model = LinearRegression().fit(X_filtered, y_filtered)
                y_pred = model.predict(X)
                plt.plot(df['year'], y, label=f"{nomes_bacias[b]}", marker='o', linestyle='-')
                plt.plot(df['year'], y_pred, linestyle='--')
            else:
                plt.plot(df['year'], y, label=f"{nomes_bacias[b]}", marker='o', linestyle='-') # Plot without trend if not enough data

    plt.title(f"Tendência de {nome_var} (1980–2010)", fontsize=14)
    plt.xlabel('Ano')
    plt.ylabel(nome_var)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()'''

##5. Significância das variações climáticas (1980 - 2010)

In [ ]:
'''#Cálculo da significância estatística das variações

print("Significância estatística das tendências (1980–2010):\n")

for var, nome_var in variaveis.items():
    print(f"Variável: {nome_var}")
    for b in bacias:
        df = dfs[b]
        x = df['year'].values
        y = df[var].values
        result = linregress(x, y)
        slope = result.slope
        p_value = result.pvalue
        r2 = result.rvalue**2
        sig = "significativa" if p_value < 0.05 else "não significativa"
        print(f"  - {nomes_bacias[b]}: inclinação = {slope:.3f}, p = {p_value:.3f}, R² = {r2:.2f} → {sig}")
    print()'''

##6. Correlação entre tendências de variáveis climáticas (1980 - 2010)

In [ ]:
'''# Etapa 4: Correlação entre tendências de variáveis climáticas
print("\nCorrelação entre variáveis climáticas agregadas (1980–2010):")

for b in bacias:
    df = dfs[b].copy()
    df_corr = df[['tmin', 'tmax', 'p', 'pet_pm']].corr(method='spearman')
    print(f"\nMatriz de correlação - {nomes_bacias[b]}:")
    print(df_corr.round(2))
    plt.figure(figsize=(6, 4))
    sns.heatmap(df_corr, annot=True, vmin=-1, vmax=1)
    plt.title(f"Correlação entre variáveis climáticas - {nomes_bacias[b]}")
    plt.tight_layout()
    plt.show()'''